In [102]:
### General Axissymmetric element

Here is a step-by-step outline of the numerical procedure for this type of analysis.

***

### ## 1. Pre-processing

* **Create 2D Mesh:** First, create a standard 2D finite element mesh of the structure's cross-section in the $r-z$ plane.
* **Decompose the Load:** Analyze your applied external load and decompose it into a Fourier series around the circumference ($\theta$). This gives you the load amplitude functions (e.g., $p_{rn}(r, z)$) for each required harmonic $n$.
* **Compute Fundamental Matrices:** For each element in the mesh, compute the three fundamental, harmonic-independent stiffness matrices: $[K_0]$, $[K_1]$, and $[K_2]$. This is done only once.

***

### ## 2. Analysis Loop 🔄

You then loop through each harmonic $n$ that is significant for your decomposed load (e.g., $n=0, 1, 2, ... N$).

**For each harmonic $n$ in the loop:**

1.  **Initialize Global Matrices:** Create empty global matrices for the stiffness $[K_n]$ and force vector $\{F_n\}$.
2.  **Element Loop:** For each element in the 2D mesh:
    * **Assemble Element Stiffness:** Calculate the element stiffness matrix for the current harmonic using the pre-computed base matrices: $[k_e]_n = [K_0] + n[K_1] + n^2[K_2]$.
    * **Calculate Element Forces:** Calculate the element's nodal force vector $\{F_n\}_e$ by integrating the load amplitudes for harmonic $n$ with the element's shape functions.
    * **Assemble into Global:** Add the element matrices $[k_e]_n$ and $\{F_n\}_e$ into the appropriate positions in the global matrices $[K_n]$ and $\{F_n\}$.
3.  **Apply Boundary Conditions:** Modify the global system to account for any prescribed zero or non-zero displacements for the current harmonic $n$.
4.  **Solve System:** Solve the linear system of equations $[K_n]\{u_n\} = \{F_n\}$ to find the vector of nodal displacement amplitudes $\{u_n\}$ for this harmonic.
5.  **Store Results:** Save the resulting displacement amplitudes $\{u_n\}$ and, if desired, calculate and save the stress amplitudes for this harmonic.

***

### ## 3. Post-processing

* **Reconstruct Solution:** After the analysis loop is complete, the final 3D displacement and stress fields are obtained by summing (superimposing) the results from each individual harmonic at any desired point $(r, \theta, z)$.

In [103]:
import pprint

import numpy as np
import sympy as sp
sp.init_printing()

### The physical properties of the element

In [104]:
ND = 3  # number of degrees of freedom per node
NNODE = 4  # number of nodes
if NNODE == 3:
    NGS = [1, 3]  # number of Gauss points, can be 1, 3
elif NNODE == 4:
    NGS = [1, 4]  # number of Gauss points, can be 1, 2, 3, 4, 5, or 6
else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

In [105]:
### The physical properties of the element
E = 200e9  # Young's modulus in Pa
nu = 0.3  # Poisson's ratio

# Define symbolic physical coordinates
r1, z1, th1, r2, z2, th2, r3, z3, th3, r4, z4, th4 = sp.symbols('r1 z1 th1 r2 z2 th2 r3 z3 th3 r4 z4 th4')

# Define symbolic variables. Capital letters R, Z are the symbols for the variables r, z
R, Z, TH = sp.symbols('R Z TH')  # radial [m], vertical [m] and circumferential [°] coordinates

if NNODE == 3:
    # Example coordinates for the nodes
    # physical_coords = {'P1': (0, 0, 0),
    #                    'P2': (1, 0, 0),
    #                    'P3': (0, 1, 0)}  # example coordinates for the nodes
    physical_coords = {'P1': (3, 0, 0),
                       'P2': (4, 0, 0),
                       'P3': (3, 0, 1)}  # example coordinates for the nodes

    midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0]) / 3, \
                (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1]) / 3, \
        (physical_coords['P1'][2] + physical_coords['P2'][2] + physical_coords['P3'][2]) / 3

    physical_coords_map = {'r1': physical_coords['P1'][0], 'th1': physical_coords['P1'][1], 'z1': physical_coords['P1'][2],
                           'r2': physical_coords['P2'][0], 'th2': physical_coords['P2'][1], 'z2': physical_coords['P2'][2],
                           'r3': physical_coords['P3'][0], 'th3': physical_coords['P3'][1], 'z3': physical_coords['P3'][2],}
    # Define the physical nodes as a matrix so it can be used a symbolic evaluation
    physical_nodes = sp.Matrix([[r1, th1, z1],
                                [r2, th2, z2],
                                [r3, th3, z3]])


elif NNODE == 4:

    # physical_coords = {'P1': (3, 0, 0),
    #                    'P2': (4, 0, 0),
    #                    'P3': (4, 0, 1),
    #                    'P4': (3, 0, 1),
    #                    }  # example coordinates for the nodes
    physical_coords = {'P1': (0.11, 0, 0),
                       'P2': (0.20, 0, 0),
                       'P3': (0.20, 0, 0.04),
                       'P4': (0.11, 0, 0.04),
                       }  # example coordinates for the nodes

    midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0] + physical_coords['P4'][0]) / 4, \
                (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1] + physical_coords['P4'][1]) / 4, \
        (physical_coords['P1'][2] + physical_coords['P2'][2] + physical_coords['P3'][2] + physical_coords['P4'][2]) / 4

    physical_coords_map = {'r1': physical_coords['P1'][0], 'th1': physical_coords['P1'][1], 'z1': physical_coords['P1'][2],
                           'r2': physical_coords['P2'][0], 'th2': physical_coords['P2'][1], 'z2': physical_coords['P2'][2],
                           'r3': physical_coords['P3'][0], 'th3': physical_coords['P3'][1], 'z3': physical_coords['P3'][2],
                           'r4': physical_coords['P4'][0], 'th4': physical_coords['P4'][1], 'z4': physical_coords['P4'][2],
                           }
    # Define the physical nodes as a matrix so it can be used a symbolic evaluation
    physical_nodes = sp.Matrix([[r1, th1, z1],
                                [r2, th2, z2],
                                [r3, th3, z3],
                                [r4, th4, z4]])

    #
    # raise NotImplementedError
    # # Example coordin
    # # physical_coords = {'P1': (0.02, 0),
    # #                    'P2': (0.11, 0),
    # #                    'P3': (0.11, 0.04),
    # #                    'P4': (0.02, 0.04),
    # #                    }  # example coordinates for the nodesates for the nodes
    # physical_coords = {'P1': (0.11, 0),
    #                    'P2': (0.20, 0),
    #                    'P3': (0.20, 0.04),
    #                    'P4': (0.11, 0.04),
    #                    }  # example coordinates for the nodesates for the nodes
    #
    # midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0] + physical_coords['P4'][0]) / 4, \
    #             (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1] + physical_coords['P4'][1]) / 4
    #
    # physical_coords_map = {'r1': physical_coords['P1'][0], 'z1': physical_coords['P1'][1],
    #                        'r2': physical_coords['P2'][0], 'z2': physical_coords['P2'][1],
    #                        'r3': physical_coords['P3'][0], 'z3': physical_coords['P3'][1],
    #                        'r4': physical_coords['P4'][0], 'z4': physical_coords['P4'][1]
    #                        }
    # # Define the physical nodes as a matrix so it can be used a symbolic evaluation
    # physical_nodes = sp.Matrix([[r1, z1],
    #                             [r2, z2],
    #                             [r3, z3],
    #                             [r4, z4],
    #                             ])


else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

In [106]:
# Define the shape functions symbolically
def shape_functions() -> sp.Matrix:
    """shape functions"""

    if NNODE == 3:
        return sp.Matrix([
            1 - R - Z,
            R,
            Z,
        ])
    elif NNODE == 4:
        return sp.Matrix([
            0.25 * (1 - R) * (1 - Z),
            0.25 * (1 + R) * (1 - Z),
            0.25 * (1 + R) * (1 + Z),
            0.25 * (1 - R) * (1 + Z),
        ])
    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")


def isoparametric_mapping() -> np.array:
    """
    Isoparametric mapping natural -> physical coordinates.

    :return: functions returning the physical coordinates
    """

    sh = shape_functions()  # shape functions with substituted values
    return sp.Matrix([sp.Matrix(sh).dot(physical_nodes[:, 0]), sp.Matrix(sh).dot(physical_nodes[:, 2])])


# Calculate the derivatives by s and t, symbolically
def shape_function_derivatives() -> np.array:
    """
    Shape function derivatives.
    [0] = dNi/ds, [1] = dNi/dt
    """
    sh = shape_functions()  # N1, N2, N3, N4
    dN_ds = []
    dN_dt = []

    for i in range(len(sh)):
        dN_ds.append(sp.diff(sh[i], R))
        dN_dt.append(sp.diff(sh[i], Z))

    return dN_ds, dN_dt


print()
print('The shape functions are:')
print(shape_functions())
print()
print('The derivatives are:')
derivatives = shape_function_derivatives()
for i, dN in enumerate(derivatives[0]):
    print(f'dN{i + 1}/ds = {dN}')
for i, dN in enumerate(derivatives[1]):
    print(f'dN{i + 1}/dt = {dN}')

print()
print('The isoparametric mapping is:')
x_map = isoparametric_mapping()[0]
y_map = isoparametric_mapping()[1]
print(f'x = {x_map}')
print(f'y = {y_map}')

# Substitute the symbolic coordinates with numerical values
st_values = {
    R: 0.3333,  # example value for s
    Z: 0.3333,  # example value for t
}
print()
print('The isoparametric mapping with coordinate values substituted is:')
x_map_substituted = x_map.subs(st_values)
y_map_substituted = y_map.subs(st_values)
print(f'x = {x_map_substituted}')
print(f'y = {y_map_substituted}')

print()
print('The derivatives of the shape functions by s are:')
print(derivatives[0])  # derivatives by s

print()
print('The derivatives of the shape functions by t are:')
print(derivatives[1])  # derivatives by t



The shape functions are:
Matrix([[(0.25 - 0.25*R)*(1 - Z)], [(1 - Z)*(0.25*R + 0.25)], [(0.25*R + 0.25)*(Z + 1)], [(0.25 - 0.25*R)*(Z + 1)]])

The derivatives are:
dN1/ds = 0.25*Z - 0.25
dN2/ds = 0.25 - 0.25*Z
dN3/ds = 0.25*Z + 0.25
dN4/ds = -0.25*Z - 0.25
dN1/dt = 0.25*R - 0.25
dN2/dt = -0.25*R - 0.25
dN3/dt = 0.25*R + 0.25
dN4/dt = 0.25 - 0.25*R

The isoparametric mapping is:
x = r1*(0.25 - 0.25*R)*(1 - Z) + r2*(1 - Z)*(0.25*R + 0.25) + r3*(0.25*R + 0.25)*(Z + 1) + r4*(0.25 - 0.25*R)*(Z + 1)
y = z1*(0.25 - 0.25*R)*(1 - Z) + z2*(1 - Z)*(0.25*R + 0.25) + z3*(0.25*R + 0.25)*(Z + 1) + z4*(0.25 - 0.25*R)*(Z + 1)

The isoparametric mapping with coordinate values substituted is:
x = 0.1111222225*r1 + 0.2222277775*r2 + 0.4444222225*r3 + 0.2222277775*r4
y = 0.1111222225*z1 + 0.2222277775*z2 + 0.4444222225*z3 + 0.2222277775*z4

The derivatives of the shape functions by s are:
[0.25*Z - 0.25, 0.25 - 0.25*Z, 0.25*Z + 0.25, -0.25*Z - 0.25]

The derivatives of the shape functions by t are:
[0.25*

### The shape function matrix

The shape function matrix $N$ is a matrix that contains the shape functions for each node. It has the structure:
$$
N = \begin{bmatrix}
N_i & 0 & 0 & ... & N_m & 0 & 0 \\
0 & N_i & 0 & ... & 0 & N_m & 0 \\
0 & 0 & N_i & ... & 0 & 0 & N_m \\
\end{bmatrix}
$$

In [107]:
def N_matrix() -> np.array:
    """
    shape function matrix N.

    """
    _N = shape_functions()  # the shape functions

    # preparing the return matrix
    _ret = sp.Matrix(ND, ND * NNODE, lambda i, j: 0)  # ND rows, ND * NNODE columns

    # filling the matrix
    for i in range(NNODE):
        for j in range(ND):
            _ret[j, j + ND * i] = _N[i]

    return _ret


_N = N_matrix()
print("The shape function matrix N is {}:".format(_N.shape))
print()
print('The shape function matrix N for this element:')
_N_e = _N.subs(physical_coords_map)
sp.pprint(_N_e)

print('\nThe shape function matrix N for this element at node 1:')
_N_e_node1 = _N_e.subs({'R': midpoint[0], 'Z': midpoint[1]})
sp.pprint(_N_e_node1.applyfunc(lambda x: x.round(4)))

# summing each row of the matrix
for i in range(ND):
    print(f'Sum of row {i + 1} of the shape function matrix N: {sum(_N_e_node1[i, :].applyfunc(lambda x: x.round(4)))}')


The shape function matrix N is (3, 12):

The shape function matrix N for this element:
⎡(0.25 - 0.25⋅R)⋅(1 - Z)             0                        0             (1 ↪
⎢                                                                              ↪
⎢           0             (0.25 - 0.25⋅R)⋅(1 - Z)             0                ↪
⎢                                                                              ↪
⎣           0                        0             (0.25 - 0.25⋅R)⋅(1 - Z)     ↪

↪  - Z)⋅(0.25⋅R + 0.25)             0                        0             (0. ↪
↪                                                                              ↪
↪          0             (1 - Z)⋅(0.25⋅R + 0.25)             0                 ↪
↪                                                                              ↪
↪          0                        0             (1 - Z)⋅(0.25⋅R + 0.25)      ↪

↪ 25⋅R + 0.25)⋅(Z + 1)             0                        0             (0.2 ↪
↪                   

### Jacobian matrix

In [108]:
def jacobian() -> sp.Matrix:
    """
    Calculate the jacobian matrix for the isoparametric mapping.
    """
    # The isoparametric mapping functions r(R,Z) and z(R,Z)
    mapping = isoparametric_mapping()
    r_map = mapping[0]
    z_map = mapping[1]

    # Calculate the derivatives
    dr_dR = sp.diff(r_map, R)
    dr_dZ = sp.diff(r_map, Z)
    dz_dR = sp.diff(z_map, R)
    dz_dZ = sp.diff(z_map, Z)

    return sp.Matrix([[dr_dR, dr_dZ],
                      [dz_dR, dz_dZ]])


# Calculate the jacobian matrix symbolically
J = jacobian()

# Substitute the symbolic coordinates with numerical values in the Jacobian
J_substituted = J.subs(physical_coords_map)

print('The jacobian matrix with coordinate values substituted is:')
sp.pprint(J_substituted)
print()

# Evaluate the jacobian matrix at some points
for r_eval in [0, 0.3333]:
    for z_eval in [0, 0.3333]:
        J_evaluated = J_substituted.subs({R: r_eval, Z: z_eval})
        print(f'Jacobian matrix at r={r_eval}, z={z_eval}:')
        sp.pprint(J_evaluated)
        print(f'The determinant at this point is |J|={J_evaluated.det():.2g}')
        print()

The jacobian matrix with coordinate values substituted is:
⎡0.045  3.46944695195361e-18⎤
⎢                           ⎥
⎣  0            0.02        ⎦

Jacobian matrix at r=0, z=0:
⎡0.045  3.46944695195361e-18⎤
⎢                           ⎥
⎣  0            0.02        ⎦
The determinant at this point is |J|=0.00090

Jacobian matrix at r=0, z=0.3333:
⎡0.045  3.46944695195361e-18⎤
⎢                           ⎥
⎣  0            0.02        ⎦
The determinant at this point is |J|=0.00090

Jacobian matrix at r=0.3333, z=0:
⎡0.045  3.46944695195361e-18⎤
⎢                           ⎥
⎣  0            0.02        ⎦
The determinant at this point is |J|=0.00090

Jacobian matrix at r=0.3333, z=0.3333:
⎡0.045  3.46944695195361e-18⎤
⎢                           ⎥
⎣  0            0.02        ⎦
The determinant at this point is |J|=0.00090



# Strain vectors for $n=0$ and $n>0$ harmonics

### Recap: Strain Vector in the Axisymmetric Case 🔄

Here is the strain vector for the simple axisymmetric case ($n=0$) with $n$ being the number of the harmonic used.

The key simplification here is that displacements do not depend on the circumferential angle $\theta$, which means all partial derivatives with respect to $\theta$ are zero ($\frac{\partial}{\partial \theta} = 0$).

We start with the general strain-displacement equations and apply the axisymmetric simplification.

The displacement field is:
$$u_r = u_r(r, z)$$$$u_\theta = u_\theta(r, z)$$$$u_z = u_z(r, z)$$
Applying the condition $\frac{\partial}{\partial \theta} = 0$ to the general strain equations gives us the following components:

* **Radial Strain ($\epsilon_r$):**
    $$\epsilon_r = \frac{\partial u_r}{\partial r}$$

* **Hoop Strain ($\epsilon_\theta$):**
    $$\epsilon_\theta = \frac{1}{r}\underbrace{\frac{\partial u_\theta}{\partial \theta}}_{0} + \frac{u_r}{r} = \frac{u_r}{r}$$

* **Axial Strain ($\epsilon_z$):**
    $$\epsilon_z = \frac{\partial u_z}{\partial z}$$

* **Torsional Shear ($\gamma_{r\theta}$):**
    $$\gamma_{r\theta} = \frac{1}{r}\underbrace{\frac{\partial u_r}{\partial \theta}}_{0} + \frac{\partial u_\theta}{\partial r} - \frac{u_\theta}{r} = \frac{\partial u_\theta}{\partial r} - \frac{u_\theta}{r}$$

* **Torsional Shear ($\gamma_{\theta z}$):**
    $$\gamma_{\theta z} = \frac{\partial u_\theta}{\partial z} + \frac{1}{r}\underbrace{\frac{\partial u_z}{\partial \theta}}_{0} = \frac{\partial u_\theta}{\partial z}$$

* **In-Plane Shear ($\gamma_{zr}$):**
    $$\gamma_{zr} = \frac{\partial u_z}{\partial r} + \frac{\partial u_r}{\partial z}$$

This conveniently decouples the problem. The final strain vector has two independent sets of components. The first is the classical plane stress situation, where no torsion is present, and the second is the torsional problem, where only the circumferential displacement $u_\theta$ is involved.

1.  **An in-plane axial/radial problem** involving only $u_r$ and $u_z$:
    $$\vec{\epsilon}_{\text{plane}} = \begin{Bmatrix} \epsilon_r \\ \epsilon_\theta \\ \epsilon_z \\ \gamma_{zr} \end{Bmatrix} = \begin{Bmatrix} \frac{\partial u_r}{\partial r} \\ \frac{u_r}{r} \\ \frac{\partial u_z}{\partial z} \\ \frac{\partial u_z}{\partial r} + \frac{\partial u_r}{\partial z} \end{Bmatrix}$$

2.  A **torsional problem** involving only the circumferential displacement $u_\theta$:
    $$\vec{\epsilon}_{\text{torsion}} = \begin{Bmatrix} \gamma_{r\theta} \\ \gamma_{\theta z} \end{Bmatrix} = \begin{Bmatrix} \frac{\partial u_\theta}{\partial r} - \frac{u_\theta}{r} \\ \frac{\partial u_\theta}{\partial z} \end{Bmatrix}$$

For harmonics where $n>0$, the situation changes because the derivatives with respect to the angle $\theta$ are no longer zero. This introduces a crucial coupling that is absent in the axisymmetric ($n=0$) case.

***

### In-Plane Strains for a General Harmonic ($n>0$)

Let's look at the strain components analogous to the plane-stress state for a general harmonic $n$. For simplicity, we'll use the displacement field for a load symmetric about the $\theta=0$ plane:
$$u_r = u_{rn}(r, z) \cos(n\theta)$$$$u_\theta = u_{\theta n}(r, z) \sin(n\theta)$$$$u_z = u_{zn}(r, z) \cos(n\theta)$$
Here is how the four strain components are derived:

* **Radial Strain ($\epsilon_r$):** This calculation is straightforward.
    $$\epsilon_r = \frac{\partial u_r}{\partial r} = \frac{\partial u_{rn}}{\partial r} \cos(n\theta)$$

* **Hoop Strain ($\epsilon_\theta$):** This is the most significant change from the $n=0$ case.
    $$\epsilon_\theta = \frac{1}{r}\frac{\partial u_\theta}{\partial \theta} + \frac{u_r}{r}$$
    The derivative of $u_\theta$ with respect to $\theta$ is now non-zero:
    $$\epsilon_\theta = \frac{1}{r}\frac{\partial}{\partial \theta}(u_{\theta n} \sin(n\theta)) + \frac{u_{rn}\cos(n\theta)}{r} = \frac{1}{r}(n u_{\theta n} \cos(n\theta)) + \frac{u_{rn}\cos(n\theta)}{r}$$   $$\epsilon_\theta = \left(\frac{n u_{\theta n} + u_{rn}}{r}\right) \cos(n\theta)$$
    Notice that the hoop strain for $n>0$ now depends on **both the radial ($u_{rn}$) and tangential ($u_{\theta n}$) displacement amplitudes**. This is a fundamental difference from the $n=0$ case where it only depended on $u_r$.

* **Axial Strain ($\epsilon_z$):** This is also straightforward.
    $$\epsilon_z = \frac{\partial u_z}{\partial z} = \frac{\partial u_{zn}}{\partial z} \cos(n\theta)$$

* **In-Plane Shear ($\gamma_{zr}$):**
    $$\gamma_{zr} = \frac{\partial u_z}{\partial r} + \frac{\partial u_r}{\partial z} = \left(\frac{\partial u_{zn}}{\partial r} + \frac{\partial u_{rn}}{\partial z}\right) \cos(n\theta)$$

So, the vector of in-plane strains for any harmonic $n>0$ looks like this:
$$\vec{\epsilon}_{\text{plane}, n} = \begin{Bmatrix} \epsilon_r \\ \epsilon_\theta \\ \epsilon_z \\ \gamma_{zr} \end{Bmatrix}_n = \begin{Bmatrix} \frac{\partial u_{rn}}{\partial r} \\ \frac{n u_{\theta n} + u_{rn}}{r} \\ \frac{\partial u_{zn}}{\partial z} \\ \frac{\partial u_{zn}}{\partial r} + \frac{\partial u_{rn}}{\partial z} \end{Bmatrix} \cos(n\theta)$$


### Torsional/Shear Strains for a General Harmonic ($n>0$)

In the axisymmetric case ($n=0$), the torsional strains only depended on the tangential displacement $u_\theta$. For any harmonic where $n>0$, this is no longer true. The radial and axial displacements now also generate shear strains due to the twisting nature of the deformation. This is a key difference.

Here are the two shear strain components that make up what you called the "torsional part" of the strain vector, $\vec{\epsilon}_{\text{torsion},n}$, for a general harmonic $n$.

* **In-Plane Shear ($\gamma_{r\theta}$):**
    This component represents the shear distortion in the $r-\theta$ plane (the cross-sectional plane).
    $$\gamma_{r\theta} = \left(\frac{\partial u_{\theta n}}{\partial r} - \frac{u_{\theta n} + n u_{rn}}{r}\right) \sin(n\theta)$$
    **Explanation:**
    * The first part, $(\frac{\partial u_{\theta n}}{\partial r} - \frac{u_{\theta n}}{r})$, is the familiar term from the $n=0$ case, representing shear from pure torsion.
    * The new term, $-\frac{n u_{rn}}{r}$, is crucial. It shows that for $n>0$, the **radial displacement ($u_{rn}$) now creates in-plane shear**. As the cross-section deforms radially in a wavy pattern, it forces the material to shear.

***

* **Transverse Shear ($\gamma_{\theta z}$):**
    This component represents the shear distortion in the $\theta-z$ plane.
    $$\gamma_{\theta z} = \left(\frac{\partial u_{\theta n}}{\partial z} - \frac{n u_{zn}}{r}\right) \sin(n\theta)$$
    **Explanation:**
    * The term $\frac{\partial u_{\theta n}}{\partial z}$ is again the familiar part from the pure torsion case at $n=0$.
    * The new term, $-\frac{n u_{zn}}{r}$, shows that the **axial displacement ($u_{zn}$) now creates transverse shear**. As the structure bends, the axial fibers stretch and compress in a wavy $\cos(n\theta)$ pattern. This differential axial movement between adjacent circumferential points induces a shearing action. This is the primary shear effect in beam bending.

In summary, for any harmonic $n>0$, the problem becomes fully coupled. Unlike the simple axisymmetric case, the radial and axial displacements now directly contribute to the shear strains, linking bending and twisting phenomena together.

***

The matrix $[B_A]$ is a **6 x (3 * m)** matrix, where `m` is the number of nodes in the element. It's composed of `m` blocks, one for each node, with each block being a 6x3 matrix.

Let's look at the computation for a single node `i`. The block $[B_{Ai}]$ is the result of multiplying the 6x3 operator $[L_A]$ by the 3x3 shape function block for node `i`, $[N_i]$:
$$[B_{Ai}] = [L_A] [N_i]$$
Where:$$[L_A] = \begin{bmatrix} \frac{\partial}{\partial r} & 0 & 0 \\ \frac{1}{r} & 0 & 0 \\ 0 & 0 & \frac{\partial}{\partial z} \\ 0 & \frac{\partial}{\partial r} - \frac{1}{r} & 0 \\ 0 & \frac{\partial}{\partial z} & 0 \\ \frac{\partial}{\partial z} & 0 & \frac{\partial}{\partial r} \end{bmatrix} \quad \text{and} \quad [N_i] = \begin{bmatrix} N_i & 0 & 0 \\ 0 & N_i & 0 \\ 0 & 0 & N_i \end{bmatrix}$$

Performing this multiplication gives the correct **6x3 matrix block** for node `i`:

$$[B_{Ai}] = \begin{bmatrix} \frac{\partial N_i}{\partial r} & 0 & 0 \\ \frac{N_i}{r} & 0 & 0 \\ 0 & 0 & \frac{\partial N_i}{\partial z} \\ 0 & \frac{\partial N_i}{\partial r} - \frac{N_i}{r} & 0 \\ 0 & \frac{\partial N_i}{\partial z} & 0 \\ \frac{\partial N_i}{\partial z} & 0 & \frac{\partial N_i}{\partial r} \end{bmatrix}$$

That operator corresponds to the following standard strain order:

1.  **Radial Strain:** $\epsilon_r$
2.  **Hoop Strain:** $\epsilon_\theta$
3.  **Axial Strain:** $\epsilon_z$
4.  **In-Plane Shear:** $\gamma_{r\theta}$
5.  **Transverse Shear:** $\gamma_{\theta z}$
6.  **Transverse Shear:** $\gamma_{zr}$

The full $[B_A]$ matrix for the element is then formed by placing these blocks side-by-side:
$$[B_A] = \begin{bmatrix} [B_{A1}] & [B_{A2}] & \cdots & [B_{Am}] \end{bmatrix}$$


### The $\begin{bmatrix}B_A\end{bmatrix}$ matrix

In [109]:
def calculate_BA():
    """
    Calculates the complete BA-matrix for the element.
    # BA relates the strain vector [ε_r, ε_θ, ε_z, γ_θr, γ_θz, γ_zr]
    """
    J_symbolic = jacobian()
    J_det_symbolic = J_symbolic.det()

    dNdr, dNdz = shape_function_derivatives()  # dN(i)/dr, dN(i)/dz

    B = sp.zeros(6, NNODE * ND)  # B matrix with 4 rows and NNODE * ND columns

    r_iso = isoparametric_mapping()[0]

    N_funcs = shape_functions()

    for i in range(NNODE):

        # ∂Nᵢ/∂r = (1/|J|) * [ (∂z/∂Z * ∂Nᵢ/∂R) - (∂r/∂Z * ∂Nᵢ/∂Z) ]
        dNi_dr = (1 / J_det_symbolic) * (sp.diff(y_map, Z) * dNdr[i] - sp.diff(x_map, Z) * dNdz[i])

        # ∂Nᵢ/∂z = (1/|J|) * [ (∂r/∂R * ∂Nᵢ/∂Z) - (∂z/∂R * ∂Nᵢ/∂R) ]
        dNi_dz = (1 / J_det_symbolic) * (sp.diff(x_map, R) * dNdz[i] - sp.diff(y_map, R) * dNdr[i])

        Ni_r = N_funcs[i] / r_iso

        B[0, ND * i] = dNi_dr
        B[1, ND * i] = Ni_r
        B[2, ND * i + 2] = dNi_dz
        B[3, ND * i + 1] = dNi_dr - Ni_r
        B[4, ND * i + 1] = dNi_dz
        B[5, ND * i] = dNi_dz
        B[5, ND * i + 2] = dNi_dr

    return B


# Example of calculating and printing the B matrix
BA_matrix = calculate_BA()

# You can then substitute values as needed
BA_evaluated = BA_matrix.subs(physical_coords_map).subs({R: 1 / 3, Z: 1 / 3, TH: 0})
# sp.pprint(BA_evaluated.evalf().applyfunc(lambda x: round(x, 3)))

# print only the first 3 columns of the matrix
print("\nThe first block of the BA matrix:")
BA_evaluated_first3 = BA_evaluated[:, :ND]
sp.pprint(BA_evaluated_first3.applyfunc(lambda x: round(x, 3)))



The first block of the BA matrix:
⎡-3.704    0       0   ⎤
⎢                      ⎥
⎢0.654     0       0   ⎥
⎢                      ⎥
⎢  0       0     -8.333⎥
⎢                      ⎥
⎢  0     -4.357    0   ⎥
⎢                      ⎥
⎢  0     -8.333    0   ⎥
⎢                      ⎥
⎣-8.333    0     -3.704⎦


### The $\begin{bmatrix}B_B\end{bmatrix}$ matrix

In [110]:
def calculate_BB():
    """
    Calculates the complete BB-matrix for the element.
    BB relates the strain vector [ε_r, ε_θ, ε_z, γ_θr, γ_θz, γ_zr]
    """
    B = sp.zeros(6, NNODE * ND)  # B matrix with 4 rows and NNODE * ND columns

    r_iso = isoparametric_mapping()[0]

    N_funcs = shape_functions()

    for i in range(NNODE):

        Ni_r = N_funcs[i] / r_iso

        B[1, ND * i + 1] = Ni_r
        B[3, ND * i] = -Ni_r
        B[4, ND * i + 2] = -Ni_r

    return B

# Example of calculating and printing the B matrix
BB_matrix = calculate_BB()
# You can then substitute values as needed
BB_evaluated = BB_matrix.subs(physical_coords_map).subs({R: 1 / 3, Z: 1 / 3, TH: 0})
# sp.pprint(BB_evaluated.evalf().applyfunc(lambda x: round(x, 3)))

# print only the first 3 columns of the matrix
print("\nThe first block of the BB matrix:")
BB_evaluated_first3 = BB_evaluated[:, :ND]
sp.pprint(BB_evaluated_first3.applyfunc(lambda x: round(x, 3)))


The first block of the BB matrix:
⎡  0       0      0   ⎤
⎢                     ⎥
⎢  0     0.654    0   ⎥
⎢                     ⎥
⎢  0       0      0   ⎥
⎢                     ⎥
⎢-0.654    0      0   ⎥
⎢                     ⎥
⎢  0       0    -0.654⎥
⎢                     ⎥
⎣  0       0      0   ⎦


### The $\begin{bmatrix}B\end{bmatrix}$ matrix

$\begin{bmatrix}B\end{bmatrix} = \begin{bmatrix}B_A\end{bmatrix} + n\begin{bmatrix}B_B\end{bmatrix}$

In [111]:
def B_matrix(n: int) -> sp.Matrix:
    """
    Calculates the complete B-matrix for the element.
    B relates the strain vector [ε_r, ε_θ, ε_z, γ_θr, γ_θz, γ_zr]
    """
    BA = calculate_BA()
    BB = calculate_BB()

    return BA + n * BB

# Example of calculating and printing the B matrix
n = 1  # harmonic number
B_matrix_result = B_matrix(n)
# You can then substitute values as needed
B_evaluated = B_matrix_result.subs(physical_coords_map).subs({R:
    1 / 3, Z: 1 / 3})
# sp.pprint(B_evaluated.evalf().applyfunc(lambda x: round(x, 3)))
# print only the first 3 columns of the matrix
print("\nThe first block of the B matrix:")
B_evaluated_first3 = B_evaluated[:, :ND]
sp.pprint(B_evaluated_first3.applyfunc(lambda x: round(x, 3)))


The first block of the B matrix:
⎡-3.704    0       0   ⎤
⎢                      ⎥
⎢0.654   0.654     0   ⎥
⎢                      ⎥
⎢  0       0     -8.333⎥
⎢                      ⎥
⎢-0.654  -4.357    0   ⎥
⎢                      ⎥
⎢  0     -8.333  -0.654⎥
⎢                      ⎥
⎣-8.333    0     -3.704⎦


### The constitutive matrix $\begin{bmatrix}D\end{bmatrix}$

The constitutive matrix relates the stress vector to the strain vector. For isotropic material it is given by:
$$
\begin{bmatrix}D\end{bmatrix} = \frac{E}{(1 - 2\nu)(1 + \nu)} \begin{bmatrix}
1 - \nu & \nu & \nu & 0 & 0 & 0 \\
\nu & 1 - \nu & \nu & 0 & 0 & 0 \\
\nu & \nu & 1 - \nu & 0 & 0 & 0 \\
0 & 0 & 0 & \frac{1 - 2\nu}{2} & 0 & 0\\
0 & 0 & 0 & 0 & \frac{1 - 2\nu}{2} & 0\\
0 & 0 & 0 & 0 & 0 & \frac{1 - 2\nu}{2}
\end{bmatrix}
$$


In [112]:
def D_matrix():
    """
    The constitutive matrix for the axissymmetric tri element.
    """
    # Plane stress condition
    D = sp.zeros(6, 6)
    D[0, 0] = 1 - nu
    D[0, 1] = nu
    D[0, 2] = nu

    D[1, 0] = nu
    D[1, 1] = 1 - nu
    D[1, 2] = nu

    D[2, 0] = nu
    D[2, 1] = nu
    D[2, 2] = 1 - nu

    D[3, 3] = D[4, 4] = D[5, 5] = (1 - 2 * nu) / 2

    D *= E / ((1 - 2 * nu) * (1 + nu))

    return D

D = D_matrix()
sp.pprint(D.applyfunc(lambda x: round(x / 1e9, 2)))

⎡269.23  115.38  115.38    0      0      0  ⎤
⎢                                           ⎥
⎢115.38  269.23  115.38    0      0      0  ⎥
⎢                                           ⎥
⎢115.38  115.38  269.23    0      0      0  ⎥
⎢                                           ⎥
⎢  0       0       0     76.92    0      0  ⎥
⎢                                           ⎥
⎢  0       0       0       0    76.92    0  ⎥
⎢                                           ⎥
⎣  0       0       0       0      0    76.92⎦


### Quadrature points and weights

In [113]:
if NNODE == 3:
    gi_data = {
        1: {
            'point': ((1/3, 1/3),),  # midpoint of the triangle
            'weight': (1,)
        },
        3: {
            'point': ((1/6, 1/6), (4/6, 1/6), (1/6, 4/6), ),
            'weight': (1/3, 1/3, 1/3)
        },
    }
elif NNODE == 4:
    gi_data = {
        1: {
            'point': [0],
            'weight': [2]
        },
        2: {
            # 'point': [-1 / np.sqrt(3), 1 / np.sqrt(3)],
            'point': [-0.5774, 0.5774],
            'weight': [1, 1]
        },
        3: {
            # 'point': [-np.sqrt(3 / 5), 0, np.sqrt(3 / 5)],
            'point': [-0.7745966, 0, 0.7745966],
            'weight': [5 / 9,  8 / 9, 5 / 9]
        },
        4: {
            'point': [-0.861136, -0.339981, 0.339981, 0.861136],
            'weight': [0.347855, 0.652145, 0.652145, 0.347855]
        },
        5: {
            'point': [-0.906180, -0.538469, 0, 0.906180, 0.538469],
            'weight': [0.236927, 0.478629, 0.568889, 0.478629, 0.236927]
        },
        6: {
            'point': [-0.932469, -0.661209, -0.238619, 0.238619, 0.661209, 0.932469],
            'weight': [0.171324, 0.360761, 0.467914, 0.467914, 0.360761, 0.171324]
        }
    }
else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

# The stiffness matrix for the element

The matrices $[K_0]$, $[K_1]$, and $[K_2]$ are the three fundamental, harmonic-independent stiffness matrices for a single element. They are calculated once and then used to instantly generate the stiffness matrix for any harmonic $n$.

Their purpose is to separate the geometry-dependent parts of the stiffness calculation from the harmonic-dependent parts.

***

# The Stiffness Matrix Formula

The element stiffness matrix for any harmonic $n$, denoted $[k_e]_n$, is constructed using this simple quadratic formula:
$$[k_e]_n = [K_0] + n [K_1] + n^2 [K_2]$$

***

# Origin of Each Matrix

These matrices arise from expanding the integral for the element stiffness matrix after separating the strain-displacement matrix $[B_n]$ into its harmonic-independent part $[B_A]$ and its linear-in-$n$ part $[B_B]$.

* **[K₀]: The Base Stiffness Matrix**
    $$[K_0] = \int_V [B_A]^T [D] [B_A] dV$$
    This matrix is derived from the parts of the strain calculation that **do not** have an explicit $n$ coefficient. It is essentially the stiffness matrix you would use for a standard axisymmetric analysis ($n=0$).

* **[K₁]: The Linear Coupling Matrix**
    $$[K_1] = \int_V ([B_A]^T [D] [B_B] + [B_B]^T [D] [B_A]) dV$$
    This matrix represents the **coupling** between the base strain terms and the harmonic-dependent strain terms. It is the part of the stiffness that grows linearly with the harmonic number $n$.

* **[K₂]: The Harmonic Stiffness Matrix**
    $$[K_2] = \int_V [B_B]^T [D] [B_B] dV$$
    This matrix is derived purely from the parts of the strain calculation that are **explicitly multiplied by $n$**. It represents the stiffness that arises solely from the non-axisymmetric deformation modes and grows with the square of the harmonic number, $n^2$.

In [114]:
if NNODE == 3:
    gi_data = {
        1: {
            'point': ((1/3, 1/3),),  # midpoint of the triangle
            'weight': (1,)
        },
        3: {
            'point': ((1/6, 1/6), (4/6, 1/6), (1/6, 4/6), ),
            'weight': (1/3, 1/3, 1/3)
        },
    }
elif NNODE == 4:
    gi_data = {
        1: {
            'point': [0],
            'weight': [2]
        },
        2: {
            # 'point': [-1 / np.sqrt(3), 1 / np.sqrt(3)],
            'point': [-0.5774, 0.5774],
            'weight': [1, 1]
        },
        3: {
            # 'point': [-np.sqrt(3 / 5), 0, np.sqrt(3 / 5)],
            'point': [-0.7745966, 0, 0.7745966],
            'weight': [5 / 9,  8 / 9, 5 / 9]
        },
        4: {
            'point': [-0.861136, -0.339981, 0.339981, 0.861136],
            'weight': [0.347855, 0.652145, 0.652145, 0.347855]
        },
        5: {
            'point': [-0.906180, -0.538469, 0, 0.906180, 0.538469],
            'weight': [0.236927, 0.478629, 0.568889, 0.478629, 0.236927]
        },
        6: {
            'point': [-0.932469, -0.661209, -0.238619, 0.238619, 0.661209, 0.932469],
            'weight': [0.171324, 0.360761, 0.467914, 0.467914, 0.360761, 0.171324]
        }
    }
else:
    raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

In [115]:
def K0_matrix(ng) -> np.array:
    """
    Gets the K0 stiffness matrix for the element.
    """
    K_lok = sp.zeros(NNODE * ND, NNODE * ND)  # initialize the stiffness matrix
    J = jacobian()
    J_substituted = J.subs(physical_coords_map)
    D = D_matrix()
    r_iso = isoparametric_mapping()[0]  # r(R, Z) mapping

    if NNODE == 3:
        for p, w in zip(gi_data[ng]['point'], gi_data[ng]['weight']):
            r_gp, z_gp = p[0], p[1]  # Gauss point in natural coords

            # Evaluate B, det(J), and r at the Gauss point
            BA_evaluated = BA_matrix.subs(physical_coords_map).subs({R: r_gp, Z: z_gp}).evalf()
            J_evaluated = J_substituted.subs({R: r_gp, Z: z_gp}).evalf()
            det_J = J_evaluated.det()
            r_physical_at_gp = r_iso.subs(physical_coords_map).subs({R: r_gp, Z: z_gp}).evalf()

            # Calculate the stiffness matrix contribution with the 2*pi*r term
            K_contribution = 2 * sp.pi * BA_evaluated.T * D * BA_evaluated * r_physical_at_gp * det_J * w
            K_lok += K_contribution

    elif NNODE == 4:

        for i in range(ng):
            for j in range(ng):
                s = gi_data[ng]['point'][i]
                t = gi_data[ng]['point'][j]
                w = gi_data[ng]['weight'][i] * gi_data[ng]['weight'][j]

                # Calculate the jacobian at the gauss point
                J_evaluated = J_substituted.subs({R: s, Z: t}).evalf()
                det_J = J_evaluated.det()
                r_physical_at_gp = r_iso.subs(physical_coords_map).subs({R: s, Z: t}).evalf()
                BA = BA_matrix.subs(physical_coords_map).subs({R: s, Z: t}).evalf()

                # Calculate the stiffness matrix contribution
                K_contribution = 2 * sp.pi * BA.T * D * BA * r_physical_at_gp * det_J * w
                K_lok += K_contribution

    else:
        raise ValueError("Unsupported number of nodes. Only 3 or 4 nodes are supported")

    return K_lok


ng = NGS[0]  # number of Gauss points, can be 1, 2, 3, 4, 5, or 6
K0 = K0_matrix(ng)
K0_1np = np.array(K0).astype(np.float64)

ng = NGS[1]
K0 = K0_matrix(ng)
K0_3np = np.array(K0).astype(np.float64)

print(np.round(K0_1np / 1e9, 1))
print()
print(np.round(K0_3np / 1e9, 1))


[[  66.5    0.    38.7   15.5    0.     1.2  -68.8    0.   -38.7  -17.8
     0.    -1.2]
 [   0.    56.     0.     0.    34.5    0.     0.   -49.8    0.     0.
   -28.3    0. ]
 [  38.7    0.   155.8  -17.5    0.   139.2  -55.     0.  -155.8    1.2
     0.  -139.2]
 [  15.5    0.   -17.5   81.     0.   -55.    -3.3    0.    17.5  -68.8
     0.    55. ]
 [   0.    34.5    0.     0.    46.3    0.     0.   -37.9    0.     0.
   -49.8    0. ]
 [   1.2    0.   139.2  -55.     0.   155.8  -17.5    0.  -139.2   38.7
     0.  -155.8]
 [ -68.8    0.   -55.    -3.3    0.   -17.5   81.     0.    55.    15.5
     0.    17.5]
 [   0.   -49.8    0.     0.   -37.9    0.     0.    46.3    0.     0.
    34.5    0. ]
 [ -38.7    0.  -155.8   17.5    0.  -139.2   55.     0.   155.8   -1.2
     0.   139.2]
 [ -17.8    0.     1.2  -68.8    0.    38.7   15.5    0.    -1.2   66.5
     0.   -38.7]
 [   0.   -28.3    0.     0.   -49.8    0.     0.    34.5    0.     0.
    56.     0. ]
 [  -1.2    0.  -139.2   